<a href="https://colab.research.google.com/github/ohhongseok/ML-DeepLearning/blob/main/4/4-1/%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이번장 키워드 및 핵심
 - 로지스틱 회귀
 - 다중분류
 - 시그모이드
 - 소프트맥스

In [2]:
import pandas as pd
#cvs 파일을 데이터 프레임을 ㅗ변환한 후, head() 로 첫번째 10개의 데이터출력
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head(10)

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
5,Bream,450.0,29.7,34.7,13.6024,4.9274
6,Bream,500.0,29.7,34.5,14.1795,5.2785
7,Bream,390.0,30.0,35.0,12.6700,4.6900
8,Bream,450.0,30.0,35.1,14.0049,4.8438
9,Bream,500.0,30.7,36.2,14.2266,4.9594


# 데이터 프레임이란?
 - 판다스에서 제공하는 2차원 표 형식의 주요 데이터 구조로, 넘파이 배열과 비슷하게 열과 행으로 이루어져 잇따.

In [9]:
# 데이터 프레임 분석 - 데이터 확인해보기
fish.describe()

,Weight,Length,Diagonal,Height,Width
count,159.000000,159.000000,159.000000,159.000000,159.000000
mean,398.326415,28.415723,31.227044,8.970994,4.417486
std,357.978317,10.716328,11.610246,4.286208,1.685804
min,0.000000,8.400000,8.800000,1.728400,1.047600
25%,120.000000,21.000000,23.150000,5.944800,3.385650
50%,273.000000,27.300000,29.400000,7.786000,4.248500
75%,650.000000,35.500000,39.650000,12.365900,5.584500
max,1650.000000,63.400000,68.000000,18.957000,8.142000


In [13]:
# 데이터 프레임 분석 - 전체적인 데이터 확인해 보기
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [14]:
# 데이터 프레임 분석 - 빈 데이터 찾기
fish.isnull().sum()

Species     0
Weight      0
Length      0
Diagonal    0
Height      0
Width       0
dtype: int64

In [16]:
# 데이터 프레임 분석 - 'Species' 열에 어떤 종류가 있는지 확인해 보기
print(pd.unique(fish['Species']))
# 이를 통해 Species 컬럼 안에 어떤 데이터들이 들어있는지 알 수 있다.

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


 - 위에서 알아본 Species 데이터를 target 데이터로 한다.
 - 외의 나머지 colunms 의 속성 ['Weight',	'Length',	'Diagonal',	'Height',	'Width'] 를 input data로 한다.

In [23]:
# input data 데이터 프레임 만들기
fish_input = fish[['Weight',	'Length',	'Diagonal',	'Height',	'Width']].to_numpy()
# 처음 데이터 5행만 출력해보기
print(fish_input[:5])
print("\n")
# 타깃데이터 만들고 확인해보기
fish_target = fish['Species'].to_numpy()
print(fish_target[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


['Bream' 'Bream' 'Bream' 'Bream' 'Bream']


In [34]:
# 데이터세트(테스트 트레이닝 데이터) 나누기 default 25% 75% 로 나뉨
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)


In [27]:
# 데이터 세트 전처리 하기
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input) # 정규화된 train input data
# 훈련세트의 통계값으로 테스트 세트를 변환해야 한다.
test_scaled = ss.transform(test_input) # 정규화된 test input data

In [31]:
print("트레이닝 데이터 자료구조 확인 : ",train_scaled.shape)
print("테스트 데이터 자료구조 확인 : ",test_scaled.shape)

트레이닝 데이터 자료구조 확인 :  (119, 5)
테스트 데이터 자료구조 확인 :  (40, 5)


In [32]:
# k 최근접 이웃 분류기의 확률 예측
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled,train_target)

print("트레이닝 점수 : ",kn.score(train_scaled,train_target))
print("테스트 점수 : ",kn.score(test_scaled,test_target))

트레이닝 점수 :  0.8907563025210085
테스트 점수 :  0.85


# 다중분류
 - fish['Species'] = 7개의 데이터 존재, 이들을 타깃데이터에 넣음
 - 이렇게 타깃데이터에 2개 이상의 클래스가 포함된 것을 ***다중분류***라고 한다.
 - 사이킷 런에서는 편리하게 이름으로 된 타깃값을 타겟데이터로 사용할 수 있다.

In [35]:
# 학습시킨 데이터 예측 결과를 테스트 데이터를 통해 알아보자
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']
